In [103]:
import numpy as np
import matplotlib.pyplot as plt
from cv2 import bitwise_and, resize
from skimage import io
from skimage.exposure import rescale_intensity, equalize_hist, equalize_adapthist
from skimage.filters import frangi

In [15]:
def contrast_stretching(img):
    p1, p99 = np.percentile(img, (1, 96))
    img = rescale_intensity(img, in_range=(p1, p99))
    return img

In [12]:
def get_blood_vessels(image, mask):

    image = np.array(image)
    mask = np.asarray(mask)
    mask = mask[:, :, 1]

    # r=0 g b=0
    image[:, :, [0, 2]] = 0
    green = image[:, :, 1]

    green = contrast_stretching(green)
    # green = equalize_hist(green)

    green = frangi(green)

    # green = contrast_stretching(green)
    green = equalize_hist(green)

    green = np.asarray(green)
    # p = 0.06  # for contrast_stretching
    p = 0.92  # for equalize_hist

    green[green > p] = 1
    green[green <= p] = 0

    res = bitwise_and(green, green, mask=mask)

    return res


In [95]:
def statistics(tp, fp, fn, tn):
    accuracy = (tp+tn)/(tp+fp+fn+tn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(fp+tn)
    
    print("Accuracy =", accuracy)
    print("Sensitivity =", sensitivity)
    print("Specificity =", specificity)    

In [96]:
def confusion_matrix(img, model):
    tp, fp, fn, tn = 0, 0, 0, 0
    for i, row in enumerate(img):
        for j, pixel in enumerate(row):
            pixel_model = model[i][j]
            # black
            if pixel == 0 and pixel == pixel_model: 
                tn += 1
            elif pixel == 0:
                fn += 1               
            # white
            elif pixel == pixel_model/255:
                tp += 1
            else:
                fp += 1
    print("tp:", tp, "| fp:", fp, "| fn:", fn, "| tn:", tn)
    statistics(tp, fp, fn, tn)

In [102]:
for number in range(1, 9):
    number = str(number)
    print('-------------')
    print('image 0' + number + '_dr')
    img = io.imread('images/0'+number+'_dr.JPG')
    mask = io.imread('mask/0'+number+'_dr_mask.tif')

    img = get_blood_vessels(img, mask)

    # fig, ax = plt.subplots()
    # ax.imshow(img, cmap="gray")
    # plt.show()

    model = io.imread('manual1/0'+number+'_dr.tif')
    confusion_matrix(img, model)

image 01_dr
Accuracy = 0.9572741231156565
Sensitivity = 0.7697234086508665
Specificity = 0.9673234683929669
image 02_dr
Accuracy = 0.9570064495762182
Sensitivity = 0.7178401848707936
Specificity = 0.9732935462883243
image 03_dr
Accuracy = 0.9524753021986614
Sensitivity = 0.7004429611650486
Specificity = 0.9686767970225761
image 04_dr
Accuracy = 0.9505776421858385
Sensitivity = 0.7022906709305269
Specificity = 0.9654359320304564
image 05_dr
Accuracy = 0.9634240173656721
Sensitivity = 0.7733870619162322
Specificity = 0.9760243323468423
image 06_dr
Accuracy = 0.9418684175032839
Sensitivity = 0.5864141392892166
Specificity = 0.9708954673600289
image 07_dr
Accuracy = 0.9586493860245825
Sensitivity = 0.6821699700954964
Specificity = 0.9819558523430634
image 08_dr
Accuracy = 0.9573457144867705
Sensitivity = 0.675233079507679
Specificity = 0.980295306191226
